# Introduction

This jupyter notebook has exploratory data analysis on the Kaggle dataset (https://www.kaggle.com/emmanuelleai/world-marathons-majors)

## Import libraries

In [177]:
!pip install pandas
!pip install altair

from datetime import datetime
import pandas as pd
import altair as alt

## Load data

In [178]:
df = pd.read_csv('world_marathon_majors.csv', encoding='ISO-8859-1')
df.head()

,year,winner,gender,country,time,marathon
0,2018,Dickson Chumba,Male,Kenya,02:05:30,Tokyo
1,2018,Birhane Dibaba,Female,Ethiopia,02:19:51,Tokyo
2,2017,Wilson Kipsang,Male,Kenya,02:03:58,Tokyo
3,2017,Sarah Chepchirchir,Female,Kenya,02:19:47,Tokyo
4,2016,Feyisa Lilesa,Male,Ethiopia,02:06:56,Tokyo


## Dataframe size

In [179]:
df.shape

(536, 6)

## Datatype checks

In [180]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536 entries, 0 to 535
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   year      536 non-null    int64 
 1   winner    536 non-null    object
 2   gender    536 non-null    object
 3   country   536 non-null    object
 4   time      536 non-null    object
 5   marathon  536 non-null    object
dtypes: int64(1), object(5)
memory usage: 25.2+ KB


## NA checks

In [181]:
df.isna().sum()

year        0
winner      0
gender      0
country     0
time        0
marathon    0
dtype: int64

## Converting total time from hours to minutes (object => float)

In [182]:
def str_to_time(x):
    """
    hh:mm:ss => total minutes
    e.g. 2:05:30 => 125.5
    """
    t = datetime.strptime(x, "%H:%M:%S")
    return t.hour * 60 + t.minute + (t.second / 60)
    
# test
str_to_time("02:05:30")

125.5

In [183]:
df['total_minutes'] = df['time'].apply(lambda x: str_to_time(x))
df.head()

,year,winner,gender,country,time,marathon,total_minutes
0,2018,Dickson Chumba,Male,Kenya,02:05:30,Tokyo,125.500000
1,2018,Birhane Dibaba,Female,Ethiopia,02:19:51,Tokyo,139.850000
2,2017,Wilson Kipsang,Male,Kenya,02:03:58,Tokyo,123.966667
3,2017,Sarah Chepchirchir,Female,Kenya,02:19:47,Tokyo,139.783333
4,2016,Feyisa Lilesa,Male,Ethiopia,02:06:56,Tokyo,126.933333


In [184]:
df.total_minutes.dtype

dtype('float64')

## EDA (Exploratory Data Analysis) and Visualizations

### Total wins by Country

In [185]:
country_counts = pd.DataFrame(df.groupby('country')['year'].agg('count')).reset_index()
country_counts.columns = ['Country', 'Wins']
country_counts.head()

,Country,Wins
0,Australia,3
1,Belgium,5
2,Brazil,7
3,Canada,17
4,China,1


In [186]:
max_wins = country_counts.Wins.max()

brush = alt.selection(type='interval', encodings=['x'])

bar_counts = alt.Chart().mark_bar().encode(
    x='Country:O',
    y='Wins:Q',
    opacity=alt.condition(brush, alt.OpacityValue(1), alt.OpacityValue(0.7)),
    tooltip=['Wins'],
    color=alt.condition(
        alt.datum.Wins == max_wins,
        alt.value('orange'),
        alt.value('steelblue')
    )
).add_selection(
    brush
)

mean_line = alt.Chart().mark_rule(
    color='red',    
).encode(
    y='mean(Wins):Q',
    size=alt.SizeValue(4)
).transform_filter(
    brush
)

final = alt.layer(bar_counts, mean_line, data=country_counts)
final.properties(
    width=850,
    height=500
).configure_axis(
    labelFontSize=15,
    titleFontSize=20
)

alt.LayerChart(...)

### Overall Female & Male winners ratio

In [187]:
gender_counts = pd.DataFrame(df.gender.value_counts()).reset_index()
gender_counts.columns = ['Gender', 'Wins']
gender_counts

,Gender,Wins
0,Male,303
1,Female,233


In [231]:
alt.Chart(gender_counts).mark_bar().encode(
    x='Gender:O',
    y='Wins:Q',
    tooltip=['Wins'],
    color=alt.Color('Gender')
).properties(
    width=300,
    height=350
).configure_axis(
    labelFontSize=15,
    titleFontSize=20,
    labelAngle=0
).configure_legend(
    strokeColor='gray',
    labelFontSize=15,
    padding=10,
    cornerRadius=5,
    orient='right'
)

alt.Chart(...)

### Wins by Place and Gender

In [214]:
place_gender_counts = df.groupby(['marathon', 'gender'])['year'].agg(['count']).reset_index()
place_gender_counts

,marathon,gender,count
0,Berlin,Female,44
1,Berlin,Male,44
2,Boston,Female,54
3,Boston,Male,121
4,Chicago,Female,40
5,Chicago,Male,40
6,London,Female,38
7,London,Male,39
8,NYC,Female,45
9,NYC,Male,47


In [261]:
alt.Chart(
    place_gender_counts,
    title='Total number of wins by gender grouped by marathon city',
).mark_bar().encode(
    x='gender:O',
    y='count:Q',
    tooltip=['count:Q'],
    color='gender:N',
    column='marathon:O',
).properties(
    width=150,
    height=250
).configure_axis(
    labelFontSize=15,
    labelAngle=0    
).configure_legend(
    strokeColor='gray',
    labelFontSize=15,
    padding=10,
    cornerRadius=5,
    orient='right'
).configure_title(
    fontSize=20,
    offset=10,
    orient='top',
    anchor='middle'
).configure_header(
    labelFontSize=18
)

alt.Chart(...)

### Histogram of marathon competitions

In [290]:
base_chart = alt.Chart(df)

hist_chart = base_chart.mark_bar().encode(
    x=alt.X('year:Q', bin=True, axis=None),
    y='count()',
    color='gender:N',
    tooltip=['count()']
)

mean_line = base_chart.mark_rule(color='red').encode(
    x='mean(year):Q',
    size=alt.value(5)
)

(hist_chart + mean_line).configure_legend(
    strokeColor='gray',
    labelFontSize=15,
    padding=10,
    cornerRadius=5,
    orient='right'
)

alt.LayerChart(...)

### Overview of total minutes taken by the marathon winners

In [331]:
year_avg_minutes = df.groupby('year')['total_minutes'].agg('mean').reset_index()
year_avg_minutes.head()

,year,total_minutes
0,1897,175.166667
1,1898,162.000000
2,1899,174.633333
3,1900,159.733333
4,1901,149.383333


In [407]:
interval = alt.selection_interval(
    mark=alt.BrushConfig(fill='green')
)
brush = alt.selection(type='interval', encodings=['x'])

base = alt.Chart(year_avg_minutes).mark_area().encode(
    x = 'year',
    y = 'total_minutes:Q',
).properties(
    width=600,
    height=250,
    selection=interval
)

minutes_chart = base.encode(
    alt.X('year:T', scale=alt.Scale(domain=brush)),
    tooltip='total_minutes'
)

chart_selector = base.properties(
    width=600,
    height=120
).add_selection(brush)

area_chart = alt.Chart(year_avg_minutes).mark_area(
    color='lightblue',
    interpolate='step-after',
    line=True
).encode(
    x='year',
    y='total_minutes',
    tooltip='total_minutes'
).properties(
    height=430,
    width=600
)

box_chart = alt.Chart(df).mark_boxplot().encode(
    x='gender',
    y='total_minutes:Q'
).properties(
    height=430,
    width=200
)

chart_selector | minutes_chart

alt.HConcatChart(...)

In [420]:
alt.Chart(df).mark_boxplot(
    size=50,
    extent=0.5
).encode(
    x='gender',
    y=alt.Y('total_minutes:Q', scale=alt.Scale(zero=False)),
    color=alt.Color('gender')
).properties(
    width=300,
    height=300
).configure_axis(
    labelFontSize=16,
    titleFontSize=16,
    labelAngle=0
)

alt.Chart(...)